In [1]:
import os
import pandas as pd
import numpy as np 
import regex as re
from collections import defaultdict

In [2]:
user_summary = pd.read_csv("../../files/user-summary.csv")

In [3]:
sites = pd.read_csv("../../files/sites.csv")

ADD STUFF TO GET RID OF THE USERS WE DON'T HAVE DATA FOR

In [4]:
def block_randomization(user_summary, sites, n):
    '''
    Combined user look up table with 
    sites table to get all possible
    combinations of group/site pairs
    
    Performs block randomization on 
    group/site combos, to get n examples
    from each 
    
    Returns a dataframe with the info
    about the sampled simulated
    users
    
    ARGS:
    user_summary = user-summary.csv in 
                   dataframe form 
    
    sites = .csv file created by the 
            output of the
            create_site_look_up()
            function 
    
    n = number of examples to sample from 
        each group/site combo
    '''
    users_df = user_summary.copy()
    users_df.user_id = users_df.user_id.apply(lambda s: f'user_{s}')
    users_df['key'] = 1
    users_df = users_df[['user_id', 'group', 'key']].values
    users_df = pd.DataFrame(users_df)
    users_df.columns = ['user', 'group', 'key']
    
    sites['key'] = 1
    
    all_sites = users_df.merge(sites, on = 'key', how = 'outer')
    
    sample_df = all_sites.groupby(['suffix', 'group']).sample(n, random_state = 42)
    
    return sample_df
    

In [5]:
t = block_randomization(user_summary, sites, 4)